In [1]:
import pandas as pd
import numpy as np
#from geopy.geocoders import Nominatim
import time
import requests
import json

In [2]:
df_raw = pd.read_csv("Raw_API_propertyExtendedSearch/Raw_data_for_sales_houses_counties_March_2023.csv")

In [3]:
df_raw.columns.tolist()

['dateSold',
 'propertyType',
 'lotAreaValue',
 'address',
 'newConstructionType',
 'variableData',
 'priceChange',
 'zestimate',
 'imgSrc',
 'price',
 'unit',
 'bedrooms',
 'contingentListingType',
 'longitude',
 'latitude',
 'listingStatus',
 'zpid',
 'rentZestimate',
 'daysOnZillow',
 'bathrooms',
 'livingArea',
 'country',
 'currency',
 'lotAreaUnit',
 'hasImage',
 'listingSubType.is_newHome',
 'variableData.text',
 'variableData.type',
 'listingSubType.is_FSBA',
 'listingSubType.is_openHouse']

In [4]:
df_raw

,dateSold,propertyType,lotAreaValue,address,priceChange,zestimate,imgSrc,price,bedrooms,contingentListingType,...,hasImage,variableData.text,variableData.type,listingSubType.is_FSBA,newConstructionType,variableData,unit,listingSubType.is_newHome,listingSubType.is_openHouse,listingSubType.is_bankOwned
0,NaN,SINGLE_FAMILY,0.3860,"301 Trinity St, Bridgeport, TX 76426",-10000.0,NaN,https://photos.zillowstatic.com/fp/8262ccbfac2...,99900,3.0,NaN,...,True,"$10,000 (Jan 26)",PRICE_REDUCTION,True,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,SINGLE_FAMILY,NaN,"Homes Available Soon Plan, Runaway Bay",NaN,NaN,https://maps.googleapis.com/maps/api/staticmap...,0,NaN,NaN,...,NaN,NaN,NaN,NaN,BUILDER_PLAN,NaN,# G00X42,True,NaN,NaN
2,NaN,SINGLE_FAMILY,0.2580,"402 Decatur St, Alvord, TX 76225",NaN,NaN,https://photos.zillowstatic.com/fp/32ffef0ba0a...,139900,3.0,NaN,...,True,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,SINGLE_FAMILY,10802.8800,"905 Brown St, Newark, TX 76071",NaN,NaN,https://photos.zillowstatic.com/fp/42ec724db57...,174000,2.0,NaN,...,True,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,SINGLE_FAMILY,7013.1600,"212 S Allen St, Boyd, TX 76023",-12000.0,194228.0,https://photos.zillowstatic.com/fp/677bc38643b...,198000,3.0,NaN,...,True,"$12,000 (Jan 25)",PRICE_REDUCTION,True,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19221,NaN,SINGLE_FAMILY,19.3202,"610 Savannah Dr, Midlothian, TX 75154",425000.0,NaN,https://photos.zillowstatic.com/fp/27c3c901505...,1300000,3.0,NaN,...,True,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN
19222,NaN,SINGLE_FAMILY,1.0000,"109 Homestead Ln, Waxahachie, TX 75165",210000.0,1208800.0,https://photos.zillowstatic.com/fp/884ab63944b...,1240000,4.0,NaN,...,True,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN
19223,NaN,SINGLE_FAMILY,8712.0000,"2838 Shane Dr, Midlothian, TX 76065",-25000.0,798400.0,https://photos.zillowstatic.com/fp/45c18d7ab09...,824900,5.0,NaN,...,True,NaN,NaN,NaN,BUILDER_SPEC,NaN,NaN,True,NaN,NaN
19224,NaN,SINGLE_FAMILY,2.9850,"1108 W Lampasas St, Ennis, TX 75119",NaN,NaN,https://photos.zillowstatic.com/fp/2957a4c0688...,899900,2.0,NaN,...,True,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df_raw['lotAreaUnit'].unique()

array(['acres', nan, 'sqft'], dtype=object)

### Ensure that we look at houses that have lot Area value and price > 0

In [5]:
df_raw1 = df_raw.copy().loc[(~df_raw['lotAreaValue'].isnull()) & (df_raw['price'] != 0)]

In [6]:
df_raw1

,dateSold,propertyType,lotAreaValue,address,daysOnZillow,price,listingDateTime,longitude,latitude,contingentListingType,...,livingArea,bathrooms,lotAreaUnit,country,currency,bedrooms,hasImage,listingSubType.is_newHome,listingSubType.is_FSBA,listingSubType.is_openHouse
1,NaN,SINGLE_FAMILY,0.700,"2104 16th St, Bridgeport, TX 76426",-1,145000,NaN,-97.750490,33.218815,NaN,...,1072.0,1.0,acres,USA,USD,2.0,True,NaN,True,NaN
2,NaN,SINGLE_FAMILY,3.000,"Private Road 3416 #3416-4, Bridgeport, TX 76426",-1,150000,NaN,-97.762085,33.190360,NaN,...,1690.0,0.0,acres,USA,USD,0.0,NaN,NaN,True,NaN
3,NaN,SINGLE_FAMILY,0.308,"509 Pettit Dr, Newark, TX 76071",-1,160000,NaN,-97.488280,32.999400,NaN,...,1316.0,1.0,acres,USA,USD,3.0,True,NaN,True,NaN
4,NaN,SINGLE_FAMILY,0.330,"813 E Highway 114, Boyd, TX 76023",-1,199000,NaN,-97.649800,33.098255,NaN,...,2849.0,2.0,acres,USA,USD,3.0,True,NaN,True,NaN
5,NaN,SINGLE_FAMILY,0.620,"105 Private Road 4768, Boyd, TX 76023",-1,223200,NaN,-97.554050,33.008650,NaN,...,1344.0,2.0,acres,USA,USD,3.0,True,NaN,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11813,NaN,SINGLE_FAMILY,5.130,"102 Old Pecan, Ennis, TX 75119",-1,820000,NaN,-96.570440,32.465630,NaN,...,1579.0,2.0,acres,USA,USD,3.0,True,NaN,True,NaN
11814,NaN,SINGLE_FAMILY,45.000,"145 Hartsfield Dr, Waxahachie, TX 75165",-1,1250000,NaN,NaN,NaN,NaN,...,1680.0,2.0,acres,USA,USD,2.0,True,NaN,True,NaN
11815,NaN,SINGLE_FAMILY,15.280,"1109 Fm 879, Waxahachie, TX 75165",-1,1500000,NaN,-96.781100,32.388800,NaN,...,1960.0,2.0,acres,USA,USD,3.0,True,NaN,True,NaN
11817,NaN,SINGLE_FAMILY,7405.200,"6941 Pampas Pl, Venus, TX 76084",-1,887068,NaN,-97.079860,32.497417,NaN,...,3324.0,4.0,sqft,USA,USD,4.0,True,True,NaN,NaN


In [7]:
df_raw1.loc[df_raw1['price'] != 0]

,dateSold,propertyType,lotAreaValue,address,daysOnZillow,price,listingDateTime,longitude,latitude,contingentListingType,...,livingArea,bathrooms,lotAreaUnit,country,currency,bedrooms,hasImage,listingSubType.is_newHome,listingSubType.is_FSBA,listingSubType.is_openHouse
1,NaN,SINGLE_FAMILY,0.700,"2104 16th St, Bridgeport, TX 76426",-1,145000,NaN,-97.750490,33.218815,NaN,...,1072.0,1.0,acres,USA,USD,2.0,True,NaN,True,NaN
2,NaN,SINGLE_FAMILY,3.000,"Private Road 3416 #3416-4, Bridgeport, TX 76426",-1,150000,NaN,-97.762085,33.190360,NaN,...,1690.0,0.0,acres,USA,USD,0.0,NaN,NaN,True,NaN
3,NaN,SINGLE_FAMILY,0.308,"509 Pettit Dr, Newark, TX 76071",-1,160000,NaN,-97.488280,32.999400,NaN,...,1316.0,1.0,acres,USA,USD,3.0,True,NaN,True,NaN
4,NaN,SINGLE_FAMILY,0.330,"813 E Highway 114, Boyd, TX 76023",-1,199000,NaN,-97.649800,33.098255,NaN,...,2849.0,2.0,acres,USA,USD,3.0,True,NaN,True,NaN
5,NaN,SINGLE_FAMILY,0.620,"105 Private Road 4768, Boyd, TX 76023",-1,223200,NaN,-97.554050,33.008650,NaN,...,1344.0,2.0,acres,USA,USD,3.0,True,NaN,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11813,NaN,SINGLE_FAMILY,5.130,"102 Old Pecan, Ennis, TX 75119",-1,820000,NaN,-96.570440,32.465630,NaN,...,1579.0,2.0,acres,USA,USD,3.0,True,NaN,True,NaN
11814,NaN,SINGLE_FAMILY,45.000,"145 Hartsfield Dr, Waxahachie, TX 75165",-1,1250000,NaN,NaN,NaN,NaN,...,1680.0,2.0,acres,USA,USD,2.0,True,NaN,True,NaN
11815,NaN,SINGLE_FAMILY,15.280,"1109 Fm 879, Waxahachie, TX 75165",-1,1500000,NaN,-96.781100,32.388800,NaN,...,1960.0,2.0,acres,USA,USD,3.0,True,NaN,True,NaN
11817,NaN,SINGLE_FAMILY,7405.200,"6941 Pampas Pl, Venus, TX 76084",-1,887068,NaN,-97.079860,32.497417,NaN,...,3324.0,4.0,sqft,USA,USD,4.0,True,True,NaN,NaN


## Ensure that the address make sense

In [8]:
##removes ones that said "Available"

In [9]:
df_raw1.loc[(df_raw1['address'].str.contains('Available', regex = False))]

,dateSold,propertyType,lotAreaValue,address,daysOnZillow,price,listingDateTime,longitude,latitude,contingentListingType,...,livingArea,bathrooms,lotAreaUnit,country,currency,bedrooms,hasImage,listingSubType.is_newHome,listingSubType.is_FSBA,listingSubType.is_openHouse


# Checking other variables

In [10]:
 'dateSold',
 'propertyType',
 'daysOnZillow',
 'price',
 'listingDateTime',
 'longitude',
 'latitude',
 'zpid',
 'imgSrc',
 'lotAreaUnit',
 'country',
 'currency',
 'hasImage',

('hasImage',)

In [10]:
df_raw1.contingentListingType.unique()

array([nan])

In [11]:
df_raw1.listingStatus.unique()

array(['FOR_SALE'], dtype=object)

In [12]:
##We want houses that have livingArea > 0
##If living area is equal to 0, then it's just a very new house listing
df_raw2 = df_raw1.copy().loc[(~df_raw1.livingArea.isnull()) & (df_raw1['livingArea'] != 0)]

In [13]:
df_raw2

,dateSold,propertyType,lotAreaValue,address,daysOnZillow,price,listingDateTime,longitude,latitude,contingentListingType,...,livingArea,bathrooms,lotAreaUnit,country,currency,bedrooms,hasImage,listingSubType.is_newHome,listingSubType.is_FSBA,listingSubType.is_openHouse
1,NaN,SINGLE_FAMILY,0.700,"2104 16th St, Bridgeport, TX 76426",-1,145000,NaN,-97.750490,33.218815,NaN,...,1072.0,1.0,acres,USA,USD,2.0,True,NaN,True,NaN
2,NaN,SINGLE_FAMILY,3.000,"Private Road 3416 #3416-4, Bridgeport, TX 76426",-1,150000,NaN,-97.762085,33.190360,NaN,...,1690.0,0.0,acres,USA,USD,0.0,NaN,NaN,True,NaN
3,NaN,SINGLE_FAMILY,0.308,"509 Pettit Dr, Newark, TX 76071",-1,160000,NaN,-97.488280,32.999400,NaN,...,1316.0,1.0,acres,USA,USD,3.0,True,NaN,True,NaN
4,NaN,SINGLE_FAMILY,0.330,"813 E Highway 114, Boyd, TX 76023",-1,199000,NaN,-97.649800,33.098255,NaN,...,2849.0,2.0,acres,USA,USD,3.0,True,NaN,True,NaN
5,NaN,SINGLE_FAMILY,0.620,"105 Private Road 4768, Boyd, TX 76023",-1,223200,NaN,-97.554050,33.008650,NaN,...,1344.0,2.0,acres,USA,USD,3.0,True,NaN,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11813,NaN,SINGLE_FAMILY,5.130,"102 Old Pecan, Ennis, TX 75119",-1,820000,NaN,-96.570440,32.465630,NaN,...,1579.0,2.0,acres,USA,USD,3.0,True,NaN,True,NaN
11814,NaN,SINGLE_FAMILY,45.000,"145 Hartsfield Dr, Waxahachie, TX 75165",-1,1250000,NaN,NaN,NaN,NaN,...,1680.0,2.0,acres,USA,USD,2.0,True,NaN,True,NaN
11815,NaN,SINGLE_FAMILY,15.280,"1109 Fm 879, Waxahachie, TX 75165",-1,1500000,NaN,-96.781100,32.388800,NaN,...,1960.0,2.0,acres,USA,USD,3.0,True,NaN,True,NaN
11817,NaN,SINGLE_FAMILY,7405.200,"6941 Pampas Pl, Venus, TX 76084",-1,887068,NaN,-97.079860,32.497417,NaN,...,3324.0,4.0,sqft,USA,USD,4.0,True,True,NaN,NaN


In [14]:
df_raw2.loc[(df_raw2['bathrooms'] == 0) | (df_raw2['bedrooms'] == 0)]

,dateSold,propertyType,lotAreaValue,address,daysOnZillow,price,listingDateTime,longitude,latitude,contingentListingType,...,livingArea,bathrooms,lotAreaUnit,country,currency,bedrooms,hasImage,listingSubType.is_newHome,listingSubType.is_FSBA,listingSubType.is_openHouse
2,NaN,SINGLE_FAMILY,3.000000,"Private Road 3416 #3416-4, Bridgeport, TX 76426",-1,150000,NaN,-97.762085,33.190360,NaN,...,1690.0,0.0,acres,USA,USD,0.0,NaN,NaN,True,NaN
4392,NaN,SINGLE_FAMILY,1.000000,"159 Hilltop Meadows Dr, Springtown, TX 76082",-1,176500,NaN,-97.639725,32.928696,NaN,...,930.0,2.0,acres,USA,USD,0.0,True,NaN,True,NaN
4875,NaN,SINGLE_FAMILY,33.000000,"1384 S Reno Rd, Springtown, TX 76082",-1,950000,NaN,-97.598976,32.927605,NaN,...,2131.0,0.0,acres,USA,USD,0.0,True,NaN,True,NaN
4950,NaN,SINGLE_FAMILY,5837.040000,"1101 Freeman St, Fort Worth, TX 76104",-1,145000,NaN,-97.308840,32.732994,NaN,...,1222.0,0.0,sqft,USA,USD,0.0,True,NaN,True,NaN
5580,NaN,SINGLE_FAMILY,9060.480000,"6432 Kenwick Ave, Fort Worth, TX 76116",-1,395000,NaN,-97.425740,32.731068,NaN,...,1914.0,0.0,sqft,USA,USD,3.0,True,NaN,True,NaN
5787,NaN,SINGLE_FAMILY,0.440000,"211 Broadmoor Ave, Arlington, TX 76010",-1,379900,NaN,-97.094770,32.734520,NaN,...,1232.0,0.0,acres,USA,USD,0.0,True,NaN,True,NaN
5962,NaN,SINGLE_FAMILY,9016.920000,"671 Farmstead Dr, Arlington, TX 76002",-1,468999,NaN,-97.081730,32.601902,NaN,...,2337.0,0.0,sqft,USA,USD,3.0,True,NaN,True,NaN
7220,NaN,SINGLE_FAMILY,15.760000,"7030 Nine Mile Bridge Rd, Fort Worth, TX 76135",-1,1250000,NaN,-97.517080,32.841007,NaN,...,3000.0,0.0,acres,USA,USD,0.0,True,NaN,True,NaN
7365,NaN,SINGLE_FAMILY,7535.880000,"2316 Michigan Ave, Dallas, TX 75216",-1,199900,NaN,-96.816605,32.719807,NaN,...,1378.0,2.0,sqft,USA,USD,0.0,True,NaN,True,NaN
7372,NaN,SINGLE_FAMILY,1873.080000,"1807 S Malcolm X Blvd, Dallas, TX 75226",-1,165000,NaN,-96.777510,32.777370,NaN,...,868.0,1.0,sqft,USA,USD,0.0,NaN,NaN,True,NaN


In [15]:
##Bathrooms and bedrooms
##Again, a house that has 0 bedroom or bathroom means that 1) it's a very new listing
##2) it has been damaged heavily and now under construction

df_raw3 = df_raw2.loc[(df_raw2['bathrooms'] != 0) & (df_raw2['bedrooms'] != 0)]

In [16]:
df_raw3

,dateSold,propertyType,lotAreaValue,address,daysOnZillow,price,listingDateTime,longitude,latitude,contingentListingType,...,livingArea,bathrooms,lotAreaUnit,country,currency,bedrooms,hasImage,listingSubType.is_newHome,listingSubType.is_FSBA,listingSubType.is_openHouse
1,NaN,SINGLE_FAMILY,0.700,"2104 16th St, Bridgeport, TX 76426",-1,145000,NaN,-97.75049,33.218815,NaN,...,1072.0,1.0,acres,USA,USD,2.0,True,NaN,True,NaN
3,NaN,SINGLE_FAMILY,0.308,"509 Pettit Dr, Newark, TX 76071",-1,160000,NaN,-97.48828,32.999400,NaN,...,1316.0,1.0,acres,USA,USD,3.0,True,NaN,True,NaN
4,NaN,SINGLE_FAMILY,0.330,"813 E Highway 114, Boyd, TX 76023",-1,199000,NaN,-97.64980,33.098255,NaN,...,2849.0,2.0,acres,USA,USD,3.0,True,NaN,True,NaN
5,NaN,SINGLE_FAMILY,0.620,"105 Private Road 4768, Boyd, TX 76023",-1,223200,NaN,-97.55405,33.008650,NaN,...,1344.0,2.0,acres,USA,USD,3.0,True,NaN,True,NaN
6,NaN,SINGLE_FAMILY,5009.400,"400 S Vick St, Decatur, TX 76234",-1,260000,NaN,-97.60398,33.231754,NaN,...,1470.0,1.0,sqft,USA,USD,3.0,True,NaN,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11813,NaN,SINGLE_FAMILY,5.130,"102 Old Pecan, Ennis, TX 75119",-1,820000,NaN,-96.57044,32.465630,NaN,...,1579.0,2.0,acres,USA,USD,3.0,True,NaN,True,NaN
11814,NaN,SINGLE_FAMILY,45.000,"145 Hartsfield Dr, Waxahachie, TX 75165",-1,1250000,NaN,NaN,NaN,NaN,...,1680.0,2.0,acres,USA,USD,2.0,True,NaN,True,NaN
11815,NaN,SINGLE_FAMILY,15.280,"1109 Fm 879, Waxahachie, TX 75165",-1,1500000,NaN,-96.78110,32.388800,NaN,...,1960.0,2.0,acres,USA,USD,3.0,True,NaN,True,NaN
11817,NaN,SINGLE_FAMILY,7405.200,"6941 Pampas Pl, Venus, TX 76084",-1,887068,NaN,-97.07986,32.497417,NaN,...,3324.0,4.0,sqft,USA,USD,4.0,True,True,NaN,NaN


In [17]:
##'listingSubType.is_newHome': Just under construction, A very new house. 
df_raw3[['listingSubType.is_newHome', 'address']]

,listingSubType.is_newHome,address
1,NaN,"2104 16th St, Bridgeport, TX 76426"
3,NaN,"509 Pettit Dr, Newark, TX 76071"
4,NaN,"813 E Highway 114, Boyd, TX 76023"
5,NaN,"105 Private Road 4768, Boyd, TX 76023"
6,NaN,"400 S Vick St, Decatur, TX 76234"
...,...,...
11813,NaN,"102 Old Pecan, Ennis, TX 75119"
11814,NaN,"145 Hartsfield Dr, Waxahachie, TX 75165"
11815,NaN,"1109 Fm 879, Waxahachie, TX 75165"
11817,True,"6941 Pampas Pl, Venus, TX 76084"


In [18]:
df_raw3[['listingSubType.is_newHome', 'address']].loc[df_raw3['listingSubType.is_newHome'] == True]

,listingSubType.is_newHome,address
7,True,"234 Tilley Ln, Decatur, TX 76234"
40,True,"844 Porcupine Dr, Springtown, TX 76082"
41,True,"166 Switchback Hill Rd, Newark, TX 76071"
48,True,"821 Porcupine Dr, Springtown, TX 76082"
49,True,"1233 Milfoil Dr, Justin, TX 76247"
...,...,...
11783,True,"2403 Magic Valley Ln, Cedar Hill, TX 75104"
11790,True,"2010 View Dr, Red Oak, TX 75154"
11810,True,"1901 Oak St #B066BC, Red Oak, TX 75154"
11817,True,"6941 Pampas Pl, Venus, TX 76084"


In [19]:
##FSBA: For sale by agent
##If FSBA is Na, most likely the house is new and under construction (newHome)
##newHome does not have HOA fee
df_raw3[['listingSubType.is_FSBA', 'address']]

,listingSubType.is_FSBA,address
1,True,"2104 16th St, Bridgeport, TX 76426"
3,True,"509 Pettit Dr, Newark, TX 76071"
4,True,"813 E Highway 114, Boyd, TX 76023"
5,True,"105 Private Road 4768, Boyd, TX 76023"
6,True,"400 S Vick St, Decatur, TX 76234"
...,...,...
11813,True,"102 Old Pecan, Ennis, TX 75119"
11814,True,"145 Hartsfield Dr, Waxahachie, TX 75165"
11815,True,"1109 Fm 879, Waxahachie, TX 75165"
11817,NaN,"6941 Pampas Pl, Venus, TX 76084"


In [20]:
##Open house: house that you can walk in to browse
##These houses are still for sale 
##Sold house have less descriptive details
df_raw3[['listingSubType.is_openHouse', 'address']].loc[df_raw3['listingSubType.is_openHouse'] == True]

,listingSubType.is_openHouse,address
29,True,"839 County Road 3592, Paradise, TX 76073"
168,True,"377 County Road 3201, Bridgeport, TX 76426"
330,True,"1912 Briarwyck Ct, Denton, TX 76205"
407,True,"103 N Clearwater Dr, Lewisville, TX 75077"
599,True,"4408 Lakeview Dr, Frisco, TX 75036"
...,...,...
11443,True,"170 N Armstrong Rd, Venus, TX 76084"
11622,True,"2224 Windy Ln, Waxahachie, TX 75167"
11624,True,"1401 Ethan Cir, Midlothian, TX 76065"
11638,True,"7241 Misty Glen Ln, Midlothian, TX 76065"


In [21]:
##Either for sale by agent or under new construction
df_raw3[['listingSubType.is_newHome', 'listingSubType.is_FSBA', 'address']].loc[
(df_raw3['listingSubType.is_newHome'] == True) & (df_raw3['listingSubType.is_FSBA'] == True)]

,listingSubType.is_newHome,listingSubType.is_FSBA,address


In [22]:
##Remove duplicates
df_raw3 = df_raw3.drop_duplicates(subset = ['zpid'])
df_raw3

,dateSold,propertyType,lotAreaValue,address,daysOnZillow,price,listingDateTime,longitude,latitude,contingentListingType,...,livingArea,bathrooms,lotAreaUnit,country,currency,bedrooms,hasImage,listingSubType.is_newHome,listingSubType.is_FSBA,listingSubType.is_openHouse
1,NaN,SINGLE_FAMILY,0.700,"2104 16th St, Bridgeport, TX 76426",-1,145000,NaN,-97.75049,33.218815,NaN,...,1072.0,1.0,acres,USA,USD,2.0,True,NaN,True,NaN
3,NaN,SINGLE_FAMILY,0.308,"509 Pettit Dr, Newark, TX 76071",-1,160000,NaN,-97.48828,32.999400,NaN,...,1316.0,1.0,acres,USA,USD,3.0,True,NaN,True,NaN
4,NaN,SINGLE_FAMILY,0.330,"813 E Highway 114, Boyd, TX 76023",-1,199000,NaN,-97.64980,33.098255,NaN,...,2849.0,2.0,acres,USA,USD,3.0,True,NaN,True,NaN
5,NaN,SINGLE_FAMILY,0.620,"105 Private Road 4768, Boyd, TX 76023",-1,223200,NaN,-97.55405,33.008650,NaN,...,1344.0,2.0,acres,USA,USD,3.0,True,NaN,True,NaN
6,NaN,SINGLE_FAMILY,5009.400,"400 S Vick St, Decatur, TX 76234",-1,260000,NaN,-97.60398,33.231754,NaN,...,1470.0,1.0,sqft,USA,USD,3.0,True,NaN,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11813,NaN,SINGLE_FAMILY,5.130,"102 Old Pecan, Ennis, TX 75119",-1,820000,NaN,-96.57044,32.465630,NaN,...,1579.0,2.0,acres,USA,USD,3.0,True,NaN,True,NaN
11814,NaN,SINGLE_FAMILY,45.000,"145 Hartsfield Dr, Waxahachie, TX 75165",-1,1250000,NaN,NaN,NaN,NaN,...,1680.0,2.0,acres,USA,USD,2.0,True,NaN,True,NaN
11815,NaN,SINGLE_FAMILY,15.280,"1109 Fm 879, Waxahachie, TX 75165",-1,1500000,NaN,-96.78110,32.388800,NaN,...,1960.0,2.0,acres,USA,USD,3.0,True,NaN,True,NaN
11817,NaN,SINGLE_FAMILY,7405.200,"6941 Pampas Pl, Venus, TX 76084",-1,887068,NaN,-97.07986,32.497417,NaN,...,3324.0,4.0,sqft,USA,USD,4.0,True,True,NaN,NaN


In [23]:
df_raw3['imgSrc']

1        https://photos.zillowstatic.com/fp/1085c1c36f2...
3        https://photos.zillowstatic.com/fp/c624b85cba3...
4        https://photos.zillowstatic.com/fp/a679c04e9a3...
5        https://photos.zillowstatic.com/fp/54215f2137c...
6        https://photos.zillowstatic.com/fp/79d1e89fa22...
                               ...                        
11813    https://photos.zillowstatic.com/fp/e045281054b...
11814    https://photos.zillowstatic.com/fp/02f887f32a7...
11815    https://photos.zillowstatic.com/fp/f8c0da3e580...
11817    https://photos.zillowstatic.com/fp/3d0c2875c6b...
11818    https://photos.zillowstatic.com/fp/2276d6d1199...
Name: imgSrc, Length: 7972, dtype: object

In [24]:
df_raw3.lotAreaUnit.unique()

array(['acres', 'sqft'], dtype=object)

In [25]:
def lotArea_in_sqft(area, unit):
    new_area = area
    if unit == 'acres':
        new_area = area * 43560
    return new_area

In [26]:
df_raw3['lotArea'] = df_raw3.apply(lambda x: lotArea_in_sqft(x['lotAreaValue'], x['lotAreaUnit']), axis = 1)

C:\Users\KHOATR~1\AppData\Local\Temp/ipykernel_12892/1703266645.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw3['lotArea'] = df_raw3.apply(lambda x: lotArea_in_sqft(x['lotAreaValue'], x['lotAreaUnit']), axis = 1)


In [27]:
0.870 * 43560

37897.2

In [28]:
df_raw3

,dateSold,propertyType,lotAreaValue,address,daysOnZillow,price,listingDateTime,longitude,latitude,contingentListingType,...,bathrooms,lotAreaUnit,country,currency,bedrooms,hasImage,listingSubType.is_newHome,listingSubType.is_FSBA,listingSubType.is_openHouse,lotArea
1,NaN,SINGLE_FAMILY,0.700,"2104 16th St, Bridgeport, TX 76426",-1,145000,NaN,-97.75049,33.218815,NaN,...,1.0,acres,USA,USD,2.0,True,NaN,True,NaN,30492.00
3,NaN,SINGLE_FAMILY,0.308,"509 Pettit Dr, Newark, TX 76071",-1,160000,NaN,-97.48828,32.999400,NaN,...,1.0,acres,USA,USD,3.0,True,NaN,True,NaN,13416.48
4,NaN,SINGLE_FAMILY,0.330,"813 E Highway 114, Boyd, TX 76023",-1,199000,NaN,-97.64980,33.098255,NaN,...,2.0,acres,USA,USD,3.0,True,NaN,True,NaN,14374.80
5,NaN,SINGLE_FAMILY,0.620,"105 Private Road 4768, Boyd, TX 76023",-1,223200,NaN,-97.55405,33.008650,NaN,...,2.0,acres,USA,USD,3.0,True,NaN,True,NaN,27007.20
6,NaN,SINGLE_FAMILY,5009.400,"400 S Vick St, Decatur, TX 76234",-1,260000,NaN,-97.60398,33.231754,NaN,...,1.0,sqft,USA,USD,3.0,True,NaN,True,NaN,5009.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11813,NaN,SINGLE_FAMILY,5.130,"102 Old Pecan, Ennis, TX 75119",-1,820000,NaN,-96.57044,32.465630,NaN,...,2.0,acres,USA,USD,3.0,True,NaN,True,NaN,223462.80
11814,NaN,SINGLE_FAMILY,45.000,"145 Hartsfield Dr, Waxahachie, TX 75165",-1,1250000,NaN,NaN,NaN,NaN,...,2.0,acres,USA,USD,2.0,True,NaN,True,NaN,1960200.00
11815,NaN,SINGLE_FAMILY,15.280,"1109 Fm 879, Waxahachie, TX 75165",-1,1500000,NaN,-96.78110,32.388800,NaN,...,2.0,acres,USA,USD,3.0,True,NaN,True,NaN,665596.80
11817,NaN,SINGLE_FAMILY,7405.200,"6941 Pampas Pl, Venus, TX 76084",-1,887068,NaN,-97.07986,32.497417,NaN,...,4.0,sqft,USA,USD,4.0,True,True,NaN,NaN,7405.20


In [29]:
null_index = df_raw3.loc[df_raw3['listingSubType.is_newHome'].isnull()].index

In [30]:
##Fill the null values with false
df_raw3.loc[null_index, 'listingSubType.is_newHome'] = False

In [31]:
df_raw3.loc[df_raw3['listingSubType.is_newHome'] == True]

,dateSold,propertyType,lotAreaValue,address,daysOnZillow,price,listingDateTime,longitude,latitude,contingentListingType,...,bathrooms,lotAreaUnit,country,currency,bedrooms,hasImage,listingSubType.is_newHome,listingSubType.is_FSBA,listingSubType.is_openHouse,lotArea
7,NaN,SINGLE_FAMILY,0.490,"234 Tilley Ln, Decatur, TX 76234",-1,250000,NaN,-97.489395,33.176320,NaN,...,2.0,acres,USA,USD,3.0,True,True,NaN,NaN,21344.400
40,NaN,SINGLE_FAMILY,9853.272,"844 Porcupine Dr, Springtown, TX 76082",-1,355425,NaN,NaN,NaN,NaN,...,2.0,sqft,USA,USD,3.0,True,True,NaN,NaN,9853.272
41,NaN,SINGLE_FAMILY,0.000,"166 Switchback Hill Rd, Newark, TX 76071",-1,339900,NaN,-97.438470,32.998684,NaN,...,2.0,sqft,USA,USD,3.0,True,True,NaN,NaN,0.000
48,NaN,SINGLE_FAMILY,9844.560,"821 Porcupine Dr, Springtown, TX 76082",-1,356500,NaN,NaN,NaN,NaN,...,2.0,sqft,USA,USD,3.0,True,True,NaN,NaN,9844.560
49,NaN,SINGLE_FAMILY,4617.360,"1233 Milfoil Dr, Justin, TX 76247",-1,399479,NaN,NaN,NaN,NaN,...,2.0,sqft,USA,USD,3.0,True,True,NaN,NaN,4617.360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11783,NaN,SINGLE_FAMILY,0.690,"2403 Magic Valley Ln, Cedar Hill, TX 75104",-1,1450000,NaN,-96.996760,32.546684,NaN,...,6.0,acres,USA,USD,4.0,True,True,NaN,NaN,30056.400
11790,NaN,SINGLE_FAMILY,1.039,"2010 View Dr, Red Oak, TX 75154",-1,820000,NaN,-96.779770,32.544006,NaN,...,5.0,acres,USA,USD,4.0,True,True,NaN,NaN,45258.840
11810,NaN,SINGLE_FAMILY,1.000,"1901 Oak St #B066BC, Red Oak, TX 75154",-1,815000,NaN,NaN,NaN,NaN,...,4.0,acres,USA,USD,4.0,True,True,NaN,NaN,43560.000
11817,NaN,SINGLE_FAMILY,7405.200,"6941 Pampas Pl, Venus, TX 76084",-1,887068,NaN,-97.079860,32.497417,NaN,...,4.0,sqft,USA,USD,4.0,True,True,NaN,NaN,7405.200


In [32]:
##If longitude is null, then latitue is null
df_raw3[df_raw3.longitude.isnull() | df_raw3.latitude.isnull()]

,dateSold,propertyType,lotAreaValue,address,daysOnZillow,price,listingDateTime,longitude,latitude,contingentListingType,...,bathrooms,lotAreaUnit,country,currency,bedrooms,hasImage,listingSubType.is_newHome,listingSubType.is_FSBA,listingSubType.is_openHouse,lotArea
11,NaN,SINGLE_FAMILY,1.110,"126 Dove St, Boyd, TX 76023",-1,299999,NaN,NaN,NaN,NaN,...,2.0,acres,USA,USD,3.0,True,False,True,NaN,48351.600
37,NaN,SINGLE_FAMILY,0.307,"201 Cattlemans Creek Rd, Newark, TX 76071",-1,375000,NaN,NaN,NaN,NaN,...,2.0,acres,USA,USD,3.0,True,False,True,NaN,13372.920
40,NaN,SINGLE_FAMILY,9853.272,"844 Porcupine Dr, Springtown, TX 76082",-1,355425,NaN,NaN,NaN,NaN,...,2.0,sqft,USA,USD,3.0,True,True,NaN,NaN,9853.272
47,NaN,SINGLE_FAMILY,6000.000,"237 Oldcastle Way, Haslet, TX 76052",-1,399000,NaN,NaN,NaN,NaN,...,2.0,sqft,USA,USD,4.0,True,False,True,NaN,6000.000
48,NaN,SINGLE_FAMILY,9844.560,"821 Porcupine Dr, Springtown, TX 76082",-1,356500,NaN,NaN,NaN,NaN,...,2.0,sqft,USA,USD,3.0,True,True,NaN,NaN,9844.560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11748,NaN,SINGLE_FAMILY,10672.200,"302 Calhoun St, Red Oak, TX 75154",-1,795000,NaN,NaN,NaN,NaN,...,6.0,sqft,USA,USD,6.0,True,False,True,NaN,10672.200
11767,NaN,SINGLE_FAMILY,1.019,"321 Rolling Meadows Dr, Waxahachie, TX 75165",-1,714900,NaN,NaN,NaN,NaN,...,3.0,acres,USA,USD,4.0,True,True,NaN,NaN,44387.640
11773,NaN,SINGLE_FAMILY,8712.000,"1321 Thunder Dove Dr, Venus, TX 76084",-1,795000,NaN,NaN,NaN,NaN,...,4.0,sqft,USA,USD,6.0,True,True,NaN,NaN,8712.000
11810,NaN,SINGLE_FAMILY,1.000,"1901 Oak St #B066BC, Red Oak, TX 75154",-1,815000,NaN,NaN,NaN,NaN,...,4.0,acres,USA,USD,4.0,True,True,NaN,NaN,43560.000


In [33]:
df_raw3[df_raw3.address.isnull()]

,dateSold,propertyType,lotAreaValue,address,daysOnZillow,price,listingDateTime,longitude,latitude,contingentListingType,...,bathrooms,lotAreaUnit,country,currency,bedrooms,hasImage,listingSubType.is_newHome,listingSubType.is_FSBA,listingSubType.is_openHouse,lotArea


## Create a function that transform the data from API propertyExtendedSearch

In [3]:
###Generate a new clean dataframe from the API propertyExtendedSearch dataset
###Use geopy to extract latitude and longtitude (doesn't work well/ do it later with property details)

def lotArea_in_sqft(area, unit):
    new_area = area
    if unit == 'acres':
        new_area = area * 43560
    return new_area

def houses_picker(df):
    
    #lot area is now in square feet
    ##Lot area is not null and price is not equal to 0
    df_raw1 = df.copy().loc[(~df['lotAreaValue'].isnull()) & (df['price'] != 0)]
    
     ##Houses that have addresses contains a string 'Available' are not included in the dataframe
    df_raw1 = df_raw1.loc[~(df_raw1['address'].str.contains('Available', regex = False))]
    
    ##Making sure that the properties have at least living area > 0, bathrooms, and bedrooms
    ##Or else it's just a plot of land
    df_raw2 = df_raw1.copy().loc[(~df_raw1.livingArea.isnull()) & (df_raw1['livingArea'] != 0)]
    df_raw3 = df_raw2.loc[(df_raw2['bathrooms'] != 0) & (df_raw2['bedrooms'] != 0)]
    
    ##A few zpid got duplicated so remove that (since it's the same house)
    df_raw4 = df_raw3.drop_duplicates(subset = ['zpid'])
    
    ##Convert houses measured in acres into square feet
    df_raw4['lotArea'] = df_raw4.apply(lambda x: lotArea_in_sqft(x['lotAreaValue'], x['lotAreaUnit']), axis = 1)
    
    ##Fill the null listingSubType with value False if they are not in one of these categories
    null_index1 = df_raw4.loc[df_raw4['listingSubType.is_newHome'].isnull()].index
    df_raw4.loc[null_index1, 'listingSubType.is_newHome'] = False
    null_index2 = df_raw4.loc[df_raw4['listingSubType.is_FSBA'].isnull()].index
    df_raw4.loc[null_index2, 'listingSubType.is_FSBA'] = False
    null_index3 = df_raw4.loc[df_raw4['listingSubType.is_openHouse'].isnull()].index
    df_raw4.loc[null_index3, 'listingSubType.is_openHouse'] = False
    df_raw4.reset_index(drop = True, inplace = True)
    columns = ['propertyType', 'lotArea', 'address', 'price', 'zpid', 'livingArea', 'bathrooms', 'bedrooms', 'country', 
               'currency', 'hasImage', 'listingSubType.is_newHome', 'listingSubType.is_FSBA', 'listingSubType.is_openHouse']
    
    return df_raw4[columns]

In [4]:
df_cleaned = houses_picker(df_raw)

C:\Users\KHOATR~1\AppData\Local\Temp/ipykernel_9632/2072760288.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw4['lotArea'] = df_raw4.apply(lambda x: lotArea_in_sqft(x['lotAreaValue'], x['lotAreaUnit']), axis = 1)


In [5]:
##This data is pulled from 6/3/2022 to 6/4/2022
##This new data is pulled on 7/29/2022
##Newest data is pulled on 8/28/2022
##Newest data is pulled on 9/29/2022
##Newest data is pulled on 10/31/2022
##Newest data is pulled on 11/30/2022
##Newest data is pulled on 12/30/2022
##Newest data is pulled on 01/29/2023
##Newest data is pulled on 02/28/2023
##Newest data is pulled on 04/01/2023
df_cleaned

,propertyType,lotArea,address,price,zpid,livingArea,bathrooms,bedrooms,country,currency,hasImage,listingSubType.is_newHome,listingSubType.is_FSBA,listingSubType.is_openHouse
0,SINGLE_FAMILY,12501.72,"210 Boling St, Alvord, TX 76225",99000,78947318,1080.0,2,2.0,USA,USD,True,False,True,False
1,SINGLE_FAMILY,21780.00,"307 S Buffalo St, Chico, TX 76431",75000,220098026,1152.0,2,3.0,USA,USD,NaN,False,True,False
2,SINGLE_FAMILY,69739.56,"4654 Fm 2264, Decatur, TX 76234",164900,239956572,1620.0,2,5.0,USA,USD,True,False,True,False
3,SINGLE_FAMILY,7013.16,"212 S Allen St, Boyd, TX 76023",180000,78938508,1197.0,2,3.0,USA,USD,True,False,True,False
4,SINGLE_FAMILY,6926.04,"210 E Washington St, Alvord, TX 76225",109900,78947832,912.0,1,2.0,USA,USD,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11020,SINGLE_FAMILY,1372140.00,"3511 Fm 66, Waxahachie, TX 75167",1350000,67647237,3292.0,3,4.0,USA,USD,True,False,True,False
11021,SINGLE_FAMILY,9880.00,"4750 Brinker St, Prosper, TX 75078",1119990,2058833127,4212.0,6,5.0,USA,USD,True,True,False,False
11022,SINGLE_FAMILY,4063276.80,"2955 Fm 667, Italy, TX 76651",1000000,98910632,1422.0,1,3.0,USA,USD,True,False,True,False
11023,SINGLE_FAMILY,43690.68,"2206 Alyssum Dr, Cedar Hill, TX 75104",955000,98898464,4398.0,4,4.0,USA,USD,True,True,False,False


In [8]:
df_cleaned.to_csv("Clean_data_API_propertyExtendedSearch/March_2023/cleaned_for_sales_houses_counties_March.csv", index = False)

## Get data properties

In [10]:
df_cleaned = pd.read_csv("Clean_data_API_propertyExtendedSearch/March_2023/cleaned_for_sales_houses_counties_March.csv")

In [12]:
str(df_cleaned.zpid.tolist()[4])

'78934974'

In [11]:
api_key = "put your key here"

In [12]:
##Pass in the list of zpid's
##return a dataframe with houses properties
def properties_details(df_clean):
    
    ##Using a list to get all of the house
    all_responses = list()
    zpid_list = df_clean.zpid.tolist()
    for zpid in zpid_list:
        url = "https://zillow-com1.p.rapidapi.com/property"
        querystring = {"zpid":str(zpid)}
        headers = {
            "X-RapidAPI-Host": "zillow-com1.p.rapidapi.com",
            "X-RapidAPI-Key": api_key}
        response = requests.request("GET", url, headers=headers, params=querystring)
        data = response.json()
        all_responses.append(data)
        time.sleep(0.2)
    
    ##Transform the list into a dataframe
    df_props = pd.json_normalize(data = all_responses)
    return df_props

#### Test case

In [11]:
df_test = df_cleaned.iloc[0:4]

In [12]:
df_test

,propertyType,lotArea,address,price,zpid,livingArea,bathrooms,bedrooms,country,currency,hasImage,listingSubType.is_newHome,listingSubType.is_FSBA,listingSubType.is_openHouse
0,SINGLE_FAMILY,30492.00,"2104 16th St, Bridgeport, TX 76426",145000,78936675,1072.0,1.0,2.0,USA,USD,True,False,True,False
1,SINGLE_FAMILY,13416.48,"509 Pettit Dr, Newark, TX 76071",157500,220112341,1316.0,1.0,3.0,USA,USD,True,False,True,False
2,SINGLE_FAMILY,14374.80,"813 E Highway 114, Boyd, TX 76023",199000,220098477,2849.0,2.0,3.0,USA,USD,True,False,True,False
3,SINGLE_FAMILY,8494.20,"105 Dodge City Ct, Newark, TX 76071",295000,118592172,1330.0,2.0,3.0,USA,USD,True,False,True,False


In [9]:
properties_details(df_test)

,listingProvider,buildingPermits,propertyTaxRate,contact_recipients,longitude,countyFIPS,cityId,timeOnZillow,url,zestimate,...,listed_by.badge_type,listed_by.business_name,listed_by.rating_average,listed_by.phone.prefix,listed_by.phone.areacode,listed_by.phone.number,listed_by.zuid,listed_by.image_url,resoFacts.exteriorFeatures,solarPotential
0,None,None,1.47,"[{'agent_reason': 1, 'zpro': None, 'recent_sal...",-97.750490,48497,23820.0,3 days,/homedetails/2104-16th-St-Bridgeport-TX-76426/...,NaN,...,Listing Agent,"C-21 SUE ANN DENTON, BRIDGEPORT",5,393,940,1197,X1-ZUz3u86qwhvw21_3wgux,https://photos.zillowstatic.com/h_n/ISsv6exiiy...,NaN,NaN
1,None,None,1.47,"[{'agent_reason': 1, 'zpro': None, 'recent_sal...",-97.488280,48497,36648.0,52 days,/homedetails/509-Pettit-Dr-Newark-TX-76071/220...,157200.0,...,Listing Agent,CENTURY 21 JUDGE FITE CO.,5,567,972,8542,X1-ZUyls0x4i6mryh_7ucam,https://photos.zillowstatic.com/h_n/ISb9dhwsp6...,"[Kennel, Storage]",NaN
2,None,None,1.47,"[{'agent_reason': 1, 'zpro': None, 'recent_sal...",-97.649800,48497,3772.0,130 days,/homedetails/813-E-Highway-114-Boyd-TX-76023/2...,NaN,...,Listing Agent,"C-21 SUE ANN DENTON, BRIDGEPORT",5,393,940,1197,X1-ZUz3u86qwhvw21_3wgux,https://photos.zillowstatic.com/h_n/ISsv6exiiy...,NaN,NaN
3,None,None,1.47,"[{'agent_reason': 1, 'zpro': None, 'recent_sal...",-97.437614,48497,NaN,1 day,/homedetails/105-Dodge-City-Ct-Newark-TX-76071...,275600.0,...,Listing Agent,The Property Shop,0,657,817,7198,X1-ZU146qxgbqh1ds9_799rj,https://photos.zillowstatic.com/h_n/ISfw9hdpoi...,[Covered Patio/Porch],NaN


 #### Do this for all of the houses

In [1]:
###Let's separate the dataframe into intervals
##Since doing that the first time somehow shuts down the kernel

In [15]:
len(df_cleaned)

13488

In [19]:
df_cleaned.loc[df_cleaned['zpid'] == 27318794]

,propertyType,lotArea,address,price,zpid,livingArea,bathrooms,bedrooms,country,currency,hasImage,listingSubType.is_newHome,listingSubType.is_FSBA,listingSubType.is_openHouse
474,SINGLE_FAMILY,7492.32,"411 S Bradshaw St, Denton, TX 76205",186000,27318794,1056.0,1,3.0,USA,USD,True,False,True,False


In [13]:
##Should create some kind of function to separate this
##So pulling data would be better
##New range is updated for July

list_range = [(0, 3200), (3200, 4200), (4200, 5200), (5200, 6200), (6200, 7200), (7200, 8200), (8200, 9200), (9200, 10200),
             (10200, 11025)]

In [14]:
##Accidentally save these into November 2022
##But the usb is not updated yet, use the usb to get the data back
for i in range(len(list_range)):
    start, end = list_range[i]
    df_temp = df_cleaned[start:end]
    df_pull = properties_details(df_temp)
    df_pull.to_csv("Separate_houses_properties_pulls/March_2023/houses_properties" + "_" + str(start) + "_" + str(end) + ".csv", index = False)
    time.sleep(0.5)

In [18]:
string1 = "houses_properties" + "_" + str(1000) + "_" + str(2000) + ".csv"

In [19]:
string1

'houses_properties_1000_2000.csv'

#### Combine all the dataframes

In [15]:
all_dfs = list()
for i in range(len(list_range)):
    start, end = list_range[i]
    file = "Separate_houses_properties_pulls/March_2023/houses_properties" + "_" + str(start) + "_" + str(end) + ".csv"
    df = pd.read_csv(file, low_memory = False)
    all_dfs.append(df)

House_data = pd.concat(all_dfs, axis = 0)
House_data.reset_index(drop= True, inplace = True)

In [16]:
House_data

,listingProvider,zpid,buildingPermits,propertyTaxRate,contact_recipients,zipcode,longitude,zestimateLowPercent,cityId,timeOnZillow,...,listingProvider.agentName,listingProvider.logos,resoFacts.otherFacts,mortgageRates,message,info,address,resoFacts,listingSubType,attributionInfo
0,NaN,7.894732e+07,NaN,1.47,"[{'agent_reason': 1, 'zpro': None, 'recent_sal...",76225.0,-97.691216,NaN,34970.0,121 days,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,2.200980e+08,NaN,1.47,"[{'agent_reason': 1, 'zpro': None, 'recent_sal...",76431.0,-97.798035,NaN,37864.0,11 days,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,2.399566e+08,NaN,1.47,"[{'agent_reason': 1, 'zpro': None, 'recent_sal...",76234.0,-97.439285,5.0,NaN,36 days,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,7.893851e+07,NaN,1.47,"[{'agent_reason': 1, 'zpro': None, 'recent_sal...",76023.0,-97.565500,6.0,3772.0,134 days,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,7.894783e+07,NaN,1.47,"[{'agent_reason': 1, 'zpro': None, 'recent_sal...",76225.0,-97.691810,NaN,34970.0,67 days,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11020,NaN,6.764724e+07,NaN,1.86,"[{'agent_reason': 1, 'zpro': None, 'recent_sal...",75167.0,-96.921290,NaN,7797.0,411 days,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11021,NaN,2.058833e+09,NaN,1.86,"[{'agent_reason': 1, 'zpro': None, 'recent_sal...",75078.0,-96.878710,NaN,45308.0,22 days,...,Shirley Davis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11022,NaN,9.891063e+07,NaN,1.86,"[{'agent_reason': 1, 'zpro': None, 'recent_sal...",76651.0,-96.839130,NaN,39127.0,230 days,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11023,NaN,9.889846e+07,NaN,1.86,"[{'agent_reason': 1, 'zpro': None, 'recent_sal...",75104.0,-97.011894,5.0,37799.0,116 days,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
##Check the number of columns in each separate dataframes
##2-3 columns are missing in a few dataframes
##But combining them now, we don't miss any of them

for i in range(len(all_dfs)):
    print("Number of columns in a dataframe: {}".format(all_dfs[i].shape[1]))

Number of columns in a dataframe: 278
Number of columns in a dataframe: 275
Number of columns in a dataframe: 275
Number of columns in a dataframe: 276
Number of columns in a dataframe: 275
Number of columns in a dataframe: 276
Number of columns in a dataframe: 276
Number of columns in a dataframe: 275
Number of columns in a dataframe: 278
Number of columns in a dataframe: 276
Number of columns in a dataframe: 277
Number of columns in a dataframe: 275


In [14]:
all_dfs[1].shape[1]

276

In [17]:
list(set(all_dfs[0].columns.tolist()) - set(all_dfs[2].columns.tolist()))

['resoFacts.otherFacts', 'listingProvider.logos', 'mortgageRates']

In [18]:
list(set(all_dfs[0].columns.tolist()) - set(all_dfs[6].columns.tolist()))

['mortgageRates']

In [19]:
list(set(all_dfs[0].columns.tolist()) - set(all_dfs[1].columns.tolist()))

['resoFacts.otherFacts', 'listingProvider.logos']

In [20]:
list(set(all_dfs[0].columns.tolist()) - set(House_data.columns.tolist()))

[]

In [17]:
House_data.to_csv("Raw_House_all_properties/March_2023/houses_properties_data_March.csv", index = False)